# EcoPlot

## Setup

### Ecoscope

In [ ]:
ECOSCOPE_RAW = "https://raw.githubusercontent.com/wildlife-dynamics/ecoscope/master"

In [ ]:
%pip install \
    'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v2.10.7' &> /dev/null

In [ ]:
import os
import sys

import geopandas as gpd
import pandas as pd

import ecoscope
import ecoscope.plotting as plotting

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

## Load Sample `Trajectory`

In [ ]:
# Read csv as Pandas DataFrame
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/er_relocs.csv.zip",
    os.path.join(output_dir, "er_relocs.csv.zip"),
)

data = pd.read_csv(os.path.join(output_dir, "er_relocs.csv.zip"), header=0, index_col=0)

# Create GeoPandas DataFrame from Pandas DataFrame, parsing the geometry column
gdf = gpd.GeoDataFrame(data, geometry=gpd.GeoSeries.from_wkt(data["geometry"], crs=4326))

# Create Relocations from the GeoPandas DataFrame
relocs = ecoscope.Relocations.from_gdf(
    gdf,
)

# Create a Trajectory from the Relocations
traj = ecoscope.Trajectory.from_relocations(relocs)

### EcoPlot Example

In [ ]:
grouped1 = traj.gdf[:1000].groupby("groupby_col")
grouped2 = traj.gdf[2000:3000].groupby("groupby_col")

epdata1 = plotting.EcoPlotData(grouped1, "segment_start", "speed_kmhr", line=dict(color="blue"))
epdata2 = plotting.EcoPlotData(grouped2, "segment_start", "speed_kmhr", line=dict(color="blue"))
epdata3 = plotting.EcoPlotData(grouped2, "segment_start", "timespan_seconds", line=dict(color="red"))

plotting.ecoplot(
    [epdata1, epdata2, epdata3],
    "My Plot",
    os.path.join(output_dir, "ecoplot_example.png"),
)